# In this notebook, I'll start the process of building and refining our algorithm (read, function... it's really only a linear combination of weights and game statistics) for calculating the game-by-game value of a player.

In [2]:
% matplotlib inline
import json
import pandas as pd
import numpy as np
import copy
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

### First, we need to load all the data frames created in Init_DataCleaning.ipynb and merge them by season. We do this so that we'll have a superset of players.

In [45]:
filenames = ['BPL/BPL12_13.pkl', 'BPL/BPL13_14.pkl', 'French/French12_13.pkl', 'French/French13_14.pkl', 
             'Germany/Bundes12_13.pkl', 'Germany/Bundes13_14.pkl', 
             'Italy/Italy12_13.pkl', 'Italy/Italy13_14.pkl', 
             'Spanish/Spanish12_13.pkl', 'Spanish/Spanish13_14.pkl']

BPL12 = pd.read_pickle(filenames[0]).drop(' ', axis=1)
BPL13 = pd.read_pickle(filenames[1]).drop(' ', axis=1)
FRA12 = pd.read_pickle(filenames[2]).drop(' ', axis=1)
FRA13 = pd.read_pickle(filenames[3]).drop(' ', axis=1)
GER12 = pd.read_pickle(filenames[4]).drop(' ', axis=1)
GER13 = pd.read_pickle(filenames[5]).drop(' ', axis=1)
ITA12 = pd.read_pickle(filenames[6]).drop(' ', axis=1)
ITA13 = pd.read_pickle(filenames[7]).drop(' ', axis=1)
SPA12 = pd.read_pickle(filenames[8]).drop(' ', axis=1)
SPA13 = pd.read_pickle(filenames[9]).drop(' ', axis=1)


In [74]:
players12 = BPL12.merge(FRA12,left_index=True, right_index=True)
players12 = players12.merge(GER12,left_index=True, right_index=True)
players12 = players12.merge(ITA12,left_index=True, right_index=True)
players12 = players12.merge(SPA12,left_index=True, right_index=True)

In [75]:
players12.shape

(38, 1679)

In [76]:
players13 = BPL13.merge(FRA13,left_index=True, right_index=True)
players13 = players13.merge(GER13,left_index=True, right_index=True)
players13 = players13.merge(ITA13,left_index=True, right_index=True)
players13 = players13.merge(SPA13,left_index=True, right_index=True)

In [77]:
players13.shape

(38, 1659)